# Beyond The Crosshair

In [353]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import json
import ast

alt.data_transformers.enable("default", max_rows=None)
alt.renderers.enable("mimetype")
alt.theme.enable("fivethirtyeight")

ThemeRegistry.enable('fivethirtyeight')

## Datasets & Cleaning

### Counter-Strike Playerbase Data from SteamDB

In [354]:
cs_data = pd.read_csv("../assets/SteamDB Counter-Strike.csv")
cs2_data = pd.read_csv("../assets/SteamDB Counter-Strike 2.csv")
css_data = pd.read_csv("../assets/SteamDB Counter-Strike Source.csv")
cscz_data = pd.read_csv("../assets/SteamDB Counter-Strike Condition Zero.csv")

In [355]:
cs_data_with_dash = cs_data[cs_data["gain"] == "-"]
cs_data_with_dash

,month,peak,gain,%gain,average,average % gain
241,Jan-04,"23,241",-,-,-,-


In [356]:
cs_data[['peak', 'gain', 'average']] = cs_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
cs2_data[['peak', 'gain', 'average']] = cs2_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
css_data[['peak', 'gain', 'average']] = css_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
cscz_data[['peak', 'gain', 'average']] = cscz_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)

In [357]:
fields_game_versions = ['Counter-Strike', 'Counter-Strike 2', 'Counter-Strike Source', 'Counter-Strike Condition Zero']
fields_to_keep = ['month', 'peak', 'gain', 'average'] + fields_game_versions
cs_data['Counter-Strike'] = 'yes'
cs2_data['Counter-Strike 2'] = 'yes'
css_data['Counter-Strike Source'] = 'yes'
cscz_data['Counter-Strike Condition Zero'] = 'yes'
cs_merged = pd.concat([cs_data.iloc[1:], cs2_data.iloc[1:], css_data.iloc[1:], cscz_data.iloc[1:]], ignore_index=True)
cs_merged = cs_merged[fields_to_keep]
cs_merged.head()

,month,peak,gain,average,Counter-Strike,Counter-Strike 2,Counter-Strike Source,Counter-Strike Condition Zero
0,Aug-25,12680.0,1946.0,7007.0,yes,NaN,NaN,NaN
1,Jul-25,14626.0,1104.0,7412.0,yes,NaN,NaN,NaN
2,Jun-25,15730.0,81.0,8251.0,yes,NaN,NaN,NaN
3,May-25,15811.0,2394.0,9224.0,yes,NaN,NaN,NaN
4,Apr-25,18205.0,430.0,9653.0,yes,NaN,NaN,NaN


In [358]:
SteamDB_merged_grouped = cs_merged.groupby(cs_merged.columns[0]).sum()
SteamDB_merged_grouped[fields_game_versions] = SteamDB_merged_grouped[fields_game_versions].replace(0, "no")

SteamDB_merged_grouped = SteamDB_merged_grouped.reset_index()
SteamDB_merged_grouped["month"] = pd.to_datetime(SteamDB_merged_grouped["month"], format="%b-%y")
SteamDB_merged_grouped = SteamDB_merged_grouped.sort_values("month")
SteamDB_merged_grouped

,month,peak,gain,average,Counter-Strike,Counter-Strike 2,Counter-Strike Source,Counter-Strike Condition Zero
82,2004-01-01,23241.0,0.0,0.0,yes,no,no,no
62,2004-02-01,39709.0,16468.0,0.0,yes,no,no,no
0,2004-04-01,84182.0,35217.0,0.0,yes,no,no,yes
123,2004-06-01,80908.0,12826.0,0.0,yes,no,no,yes
103,2004-07-01,84804.0,3896.0,0.0,yes,no,no,yes
...,...,...,...,...,...,...,...,...
19,2025-04-01,1898525.0,40987.0,1070094.0,yes,yes,yes,yes
180,2025-05-01,1752347.0,146178.0,1060039.0,yes,yes,yes,yes
142,2025-06-01,1766572.0,14429.0,1029242.0,yes,yes,yes,yes
122,2025-07-01,1506027.0,338169.0,954251.0,yes,yes,yes,yes


In [359]:
# SteamDB_merged_grouped.to_csv("../assets/SteamDB_merged_cleaned.csv",encoding="utf-8")

### Rival Game Data from IGDB

In [360]:
"""
Used to split igdb data into 3 separate parts because of github upload limit, commented out after use
"""

# game_data = pd.read_csv("../assets/igdb_shooters.csv")

# splitting all genres csv into separate csv files

# Replace this with the path to your big CSV
# input_file = "../assets/igdb_games.csv"

# # Read the CSV into a DataFrame
# df = pd.read_csv(input_file)

# # Calculate the size of each split
# n = len(df)
# split_size = n // 3

# # Split into 3 parts
# df1 = df.iloc[:split_size]
# df2 = df.iloc[split_size:2*split_size]
# df3 = df.iloc[2*split_size:]

# Save to separate CSV files
# df1.to_csv("../assets/igdb_part1.csv", index=False)
# df2.to_csv("../assets/igdb_part2.csv", index=False)
# df3.to_csv("../assets/igdb_part3.csv", index=False)

# print("Splitting complete! Files saved as part1.csv, part2.csv, part3.csv")

'\nUsed to split igdb data into 3 separate parts because of github upload limit, commented out after use\n'

In [361]:
igdb_data_1 = pd.read_csv("../assets/igdb_part1.csv")
igdb_data_2 = pd.read_csv("../assets/igdb_part2.csv")
igdb_data_3 = pd.read_csv("../assets/igdb_part3.csv")
igdb_data_merged = pd.concat([igdb_data_1, igdb_data_2, igdb_data_3], ignore_index=True)

/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_3278/2886035041.py:1: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  igdb_data_1 = pd.read_csv("../assets/igdb_part1.csv")
/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_3278/2886035041.py:2: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  igdb_data_2 = pd.read_csv("../assets/igdb_part2.csv")
/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_3278/2886035041.py:3: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  igdb_data_3 = pd.read_csv("../assets/igdb_part3.csv")


In [373]:
IGDB_fields = ['id', 'name', 'first_release_date', 'genres', 'rating', 'rating_count', 'total_rating', 'total_rating_count', 'aggregated_rating',
       'aggregated_rating_count']

igdb_data_merged_filtered = igdb_data_merged[IGDB_fields]
igdb_data_merged_filtered["first_release_date"] = pd.to_datetime(igdb_data_merged_filtered["first_release_date"], unit="s") # Formatting date

/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_3278/1111918781.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  igdb_data_merged_filtered["first_release_date"] = pd.to_datetime(igdb_data_merged_filtered["first_release_date"], unit="s") # Formatting date


In [374]:
vanilla_wow = igdb_data_merged_filtered[igdb_data_merged_filtered['id']==123] # Vanilla WoW
cata_wow = igdb_data_merged_filtered[igdb_data_merged_filtered['id']==229] # Cataclysm
cata_wow

,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
327330,229,World of Warcraft: Cataclysm,2010-12-07,[12],76.861351,198.0,83.097342,201.0,89.333333,3.0


In [375]:
with open("../data/genres.json", "r", encoding="utf-8") as f:
    genres_list = json.load(f)
    genres_list = {int(k): v for k, v in genres_list.items()}

genres_list

{31: 'Adventure',
 33: 'Arcade',
 35: 'Card & Board Game',
 4: 'Fighting',
 25: "Hack and slash/Beat 'em up",
 32: 'Indie',
 36: 'MOBA',
 7: 'Music',
 30: 'Pinball',
 8: 'Platform',
 2: 'Point-and-click',
 9: 'Puzzle',
 26: 'Quiz/Trivia',
 10: 'Racing',
 11: 'Real Time Strategy (RTS)',
 12: 'Role-playing (RPG)',
 5: 'Shooter',
 13: 'Simulator',
 14: 'Sport',
 15: 'Strategy',
 24: 'Tactical',
 16: 'Turn-based strategy (TBS)',
 34: 'Visual Novel'}

In [376]:
igdb_data_merged_filtered['genres'] = igdb_data_merged_filtered['genres'].map(lambda x: [] if pd.isna(x) else[genres_list[g] for g in x] if isinstance(x, list) else [genres_list[g] for g in ast.literal_eval(x)])
igdb_data_merged_filtered.head()

/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_3278/129189592.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  igdb_data_merged_filtered['genres'] = igdb_data_merged_filtered['genres'].map(lambda x: [] if pd.isna(x) else[genres_list[g] for g in x] if isinstance(x, list) else [genres_list[g] for g in ast.literal_eval(x)])


,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
0,165499,^_^,2012-01-14,[Adventure],NaN,NaN,NaN,NaN,NaN,NaN
1,141830,||[}}}°.•°.°•..°•°[|||{{{,2020-02-02,[Indie],NaN,NaN,NaN,NaN,NaN,NaN
2,191680,_____,2014-12-11,"[Shooter, Indie]",NaN,NaN,NaN,NaN,NaN,NaN
3,176875,__________,2018-07-05,"[Adventure, Indie]",NaN,NaN,NaN,NaN,NaN,NaN
4,92204,_-_,2018-03-14,"[Puzzle, Arcade]",90.0,0.0,90.0,0.0,NaN,NaN


In [398]:
# Filtering entries without rating
print(f"With and without rating = {len(igdb_data_merged_filtered)}")
igdb_data_merged_filtered_converted = igdb_data_merged_filtered.dropna(subset=['rating'])
print(f"With rating = {len(igdb_data_merged_filtered_converted)}")

# Filtering entries without first_release_date
print(f"With and without first_release_date = {len(igdb_data_merged_filtered_converted)}")
igdb_data_merged_filtered_converted = igdb_data_merged_filtered_converted[~igdb_data_merged_filtered_converted["first_release_date"].isna()]
print(f"With first_release_date = {len(igdb_data_merged_filtered_converted)}")

# Filtering entries before 2004
igdb_data_merged_filtered_converted = igdb_data_merged_filtered_converted[pd.to_datetime(igdb_data_merged_filtered_converted["first_release_date"]) > pd.to_datetime("2004-01-01")]
igdb_data_merged_filtered_converted.head()

With and without rating = 335086
With rating = 32671
With and without first_release_date = 32671
With first_release_date = 31702


,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
4,92204,_-_,2018-03-14,"[Puzzle, Arcade]",90.000000,0.0,90.000000,0.0,NaN,NaN
23,1649,007 Legends,2012-10-16,[Shooter],54.172800,23.0,49.943543,30.0,45.714286,7.0
48,113188,03.04,2019-01-01,"[Adventure, Indie]",60.000000,2.0,60.000000,2.0,NaN,NaN
69,55486,0°N 0°W,2018-03-02,"[Adventure, Indie]",70.000000,4.0,60.000000,5.0,50.000000,1.0
72,9583,0rbitalis,2014-04-16,"[Puzzle, Simulator, Indie]",79.935551,5.0,77.467775,8.0,75.000000,3.0


In [399]:
games_with_counter = igdb_data_merged_filtered_converted[igdb_data_merged_filtered_converted["name"].str.contains("counter", case=False, na=False)]
games_with_counter

,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
58976,33278,CounterAttack,2016-04-21,"[Shooter, Adventure, Indie, Arcade]",80.000000,1.0,80.000000,1.0,NaN,NaN
58993,30643,Counter Fight,2016-11-14,"[Simulator, Indie]",80.000000,0.0,80.000000,0.0,NaN,NaN
58997,40553,Counter Fight: Samurai Edition,2017-06-07,"[Simulator, Indie]",70.000000,0.0,70.000000,0.0,NaN,NaN
59023,35703,Counter Spell,2015-10-05,"[Role-playing (RPG), Indie]",80.000000,0.0,80.000000,0.0,NaN,NaN
59025,7612,CounterSpy,2014-08-19,"[Shooter, Tactical, Indie]",75.046860,36.0,69.210930,44.0,63.375000,8.0
59028,242408,Counter-Strike 2,2023-09-27,"[Shooter, Tactical]",67.185101,249.0,67.185101,249.0,NaN,NaN
59031,297,Counter-Strike: Condition Zero,2004-03-23,[Shooter],74.174541,217.0,70.253937,220.0,66.333333,3.0
59032,93807,Counter-Strike: Condition Zero Deleted Scenes,2004-03-23,[Shooter],64.784238,32.0,64.784238,32.0,NaN,NaN
59033,1372,Counter-Strike: Global Offensive,2012-08-21,"[Shooter, Tactical]",81.813980,1245.0,81.406990,1250.0,81.000000,5.0
59037,16957,Counter-Strike Nexon,2014-10-07,"[Shooter, Indie]",20.907759,22.0,20.907759,22.0,NaN,NaN


In [400]:
# igdb_data_merged_filtered_converted.to_csv("../assets/IGDB_merged_cleaned.csv", index=False)

### Twitch Viewership Data from TwitchTracker

In [380]:
cs_twitch_data = pd.read_csv("../assets/TwitchTracker Counter-Strike.csv")
cs2_twitch_data = pd.read_csv("../assets/TwitchTracker Counter-Strike 2.csv")
cscz_twitch_data = pd.read_csv("../assets/TwitchTracker Counter-Strike Condition Zero.csv")
css_twitch_data = pd.read_csv("../assets/TwitchTracker Counter-Strike Source.csv")

twitch_fields = ['Month', 'Avg Viewers', 'Gain', 'Peak Viewers', 'Avg Streams', 'Gain.1', 'Peak Streams', 'Hours Watched']

In [381]:
twitch_data_merged = pd.concat([cs_twitch_data, cs2_twitch_data, cscz_twitch_data, css_twitch_data], ignore_index=True)
twitch_data_merged = twitch_data_merged[twitch_fields]
twitch_data_merged

,Month,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched
0,Nov-16,18.0,-,164,2.0,-,7.0,3.2K
1,Dec-16,20.0,2,509,2.0,-,10.0,8.0K
2,Jan-17,19.0,-1,543,4.0,2,13.0,8.8K
3,Feb-17,30.0,11,"5,782",2.0,-2,11.0,16.0K
4,Mar-17,18.0,-12,815,2.0,-,10.0,6.9K
...,...,...,...,...,...,...,...,...
316,Jan-17,18,-1,215,1,-,3,357
317,Dec-16,19,4,216,1,-,3,946
318,Nov-16,15,-,147,1,-,3,336
319,Aug-25,14,-,"1,213",2,-,8,4.4K


In [382]:
# Transforming Hours Watched format
def convert_k(val):
    if isinstance(val, str):
        if val.endswith("K"):
            return float(val[:-1].replace(".", ""))* 1000
        elif val.endswith("M"):
            return float(val[:-1].replace(".", "")) * 1000000
    return float(val)

twitch_data_merged["Hours Watched"] = twitch_data_merged["Hours Watched"].map(convert_k)
twitch_data_merged

,Month,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched
0,Nov-16,18.0,-,164,2.0,-,7.0,32000.0
1,Dec-16,20.0,2,509,2.0,-,10.0,80000.0
2,Jan-17,19.0,-1,543,4.0,2,13.0,88000.0
3,Feb-17,30.0,11,"5,782",2.0,-2,11.0,160000.0
4,Mar-17,18.0,-12,815,2.0,-,10.0,69000.0
...,...,...,...,...,...,...,...,...
316,Jan-17,18,-1,215,1,-,3,357.0
317,Dec-16,19,4,216,1,-,3,946.0
318,Nov-16,15,-,147,1,-,3,336.0
319,Aug-25,14,-,"1,213",2,-,8,44000.0


In [383]:
numeric_fields = [x for x in twitch_fields if x != "Month"]
twitch_data_merged[numeric_fields] = twitch_data_merged[numeric_fields].replace({",": "", "-": "0"}, regex=True).astype(float)

twitch_data_merged["Month"] = pd.to_datetime(twitch_data_merged["Month"], format="%b-%y")
twitch_data_merged = twitch_data_merged.sort_values("Month")
twitch_data_merged.head()

,Month,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched
0,2016-11-01,18.0,0.0,164.0,2.0,0.0,7.0,32000.0
318,2016-11-01,15.0,0.0,147.0,1.0,0.0,3.0,336.0
212,2016-11-01,41089.0,0.0,236838.0,800.0,0.0,1723.0,305000000.0
211,2016-12-01,97356.0,56267.0,1153507.0,937.0,137.0,2172.0,721000000.0
1,2016-12-01,20.0,2.0,509.0,2.0,0.0,10.0,80000.0


In [384]:
twitch_data_merged_grouped = twitch_data_merged.groupby(twitch_data_merged.columns[0]).sum()
# twitch_data_merged_grouped.to_csv("../assets/TwitchTracker_merged_cleaned.csv", encoding="utf-8")
twitch_data_merged_grouped

,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched
Month,,,,,,,
2016-11-01,41122.0,0.0,237149.0,803.0,0.0,1733.0,305032336.0
2016-12-01,97395.0,56273.0,1154232.0,940.0,137.0,2185.0,721080946.0
2017-01-01,60177.0,37218.0,461337.0,1014.0,74.0,2097.0,404088357.0
2017-02-01,49606.0,10593.0,512456.0,946.0,68.0,2100.0,368160397.0
2017-03-01,50868.0,1286.0,418276.0,879.0,69.0,1830.0,364069433.0
...,...,...,...,...,...,...,...
2025-04-01,99822.0,12660.0,722031.0,1601.0,80.0,3272.0,741902172.0
2025-05-01,117782.0,18074.0,1146827.0,1481.0,120.0,3053.0,848466647.0
2025-06-01,86343.0,31463.0,556026.0,1519.0,38.0,2998.0,642565317.0


### Youtube Video Data from Youtube Data API

In [385]:
youtube_data = pd.read_csv("../assets/yt_counter_strike.csv")

In [386]:
youtube_fields = ['publishedAt', 'title', 'description', 'viewCount', 'likeCount', 'commentCount', 'favoriteCount']
youtube_data = youtube_data[youtube_fields]
youtube_data["publishedAt"] = pd.to_datetime(youtube_data["publishedAt"], utc=True).dt.to_period("M").dt.to_timestamp()

youtube_data = youtube_data.sort_values("publishedAt")
youtube_data.head()

/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_3278/2866478919.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  youtube_data["publishedAt"] = pd.to_datetime(youtube_data["publishedAt"], utc=True).dt.to_period("M").dt.to_timestamp()


,publishedAt,title,description,viewCount,likeCount,commentCount,favoriteCount
46,2005-08-01,Offline Quicky,my 1st video ofline park DM me against the rest,1064,9.0,6.0,0
11,2005-08-01,Damnation,all about the map damnation,19378,19.0,4.0,0
25,2005-08-01,A tribute to the A.I.s,Just see it.\r\n\r\nP.S.it gets better as it gos.,5109,59.0,17.0,0
38,2005-08-01,Welcome To Our Arena -Trailor,"Preview of upcoming TeaM f@taL Movie ""Welcome ...",2259,28.0,12.0,0
6,2005-08-01,Black Hawk Down CTF Video 32 player online PS2,This is a video i made of Black Hawk down duri...,50844,90.0,65.0,0


In [387]:
# youtube_data.to_csv("../assets/Youtube_merged_cleaned.csv", encoding="utf-8")

In [388]:
counts = youtube_data.groupby("publishedAt").size().reset_index(name="video_count")
counts_sorted = counts.sort_values(by="video_count", ascending=False)
counts_sorted

,publishedAt,video_count
239,2025-08-01,50
210,2023-03-01,40
149,2018-01-01,23
3,2005-11-01,21
115,2015-03-01,21
...,...,...
209,2023-02-01,6
98,2013-10-01,5
162,2019-02-01,5
208,2023-01-01,4


## Visualizations

In [392]:
SteamDB_merged_grouped.head()

,month,peak,gain,average,Counter-Strike,Counter-Strike 2,Counter-Strike Source,Counter-Strike Condition Zero
82,2004-01-01,23241.0,0.0,0.0,yes,no,no,no
62,2004-02-01,39709.0,16468.0,0.0,yes,no,no,no
0,2004-04-01,84182.0,35217.0,0.0,yes,no,no,yes
123,2004-06-01,80908.0,12826.0,0.0,yes,no,no,yes
103,2004-07-01,84804.0,3896.0,0.0,yes,no,no,yes


In [401]:
igdb_data_merged_filtered_converted.head()

,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
4,92204,_-_,2018-03-14,"[Puzzle, Arcade]",90.000000,0.0,90.000000,0.0,NaN,NaN
23,1649,007 Legends,2012-10-16,[Shooter],54.172800,23.0,49.943543,30.0,45.714286,7.0
48,113188,03.04,2019-01-01,"[Adventure, Indie]",60.000000,2.0,60.000000,2.0,NaN,NaN
69,55486,0°N 0°W,2018-03-02,"[Adventure, Indie]",70.000000,4.0,60.000000,5.0,50.000000,1.0
72,9583,0rbitalis,2014-04-16,"[Puzzle, Simulator, Indie]",79.935551,5.0,77.467775,8.0,75.000000,3.0


In [394]:
twitch_data_merged_grouped.head()

,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched
Month,,,,,,,
2016-11-01,41122.0,0.0,237149.0,803.0,0.0,1733.0,305032336.0
2016-12-01,97395.0,56273.0,1154232.0,940.0,137.0,2185.0,721080946.0
2017-01-01,60177.0,37218.0,461337.0,1014.0,74.0,2097.0,404088357.0
2017-02-01,49606.0,10593.0,512456.0,946.0,68.0,2100.0,368160397.0
2017-03-01,50868.0,1286.0,418276.0,879.0,69.0,1830.0,364069433.0


In [395]:
youtube_data.head()

,publishedAt,title,description,viewCount,likeCount,commentCount,favoriteCount
46,2005-08-01,Offline Quicky,my 1st video ofline park DM me against the rest,1064,9.0,6.0,0
11,2005-08-01,Damnation,all about the map damnation,19378,19.0,4.0,0
25,2005-08-01,A tribute to the A.I.s,Just see it.\r\n\r\nP.S.it gets better as it gos.,5109,59.0,17.0,0
38,2005-08-01,Welcome To Our Arena -Trailor,"Preview of upcoming TeaM f@taL Movie ""Welcome ...",2259,28.0,12.0,0
6,2005-08-01,Black Hawk Down CTF Video 32 player online PS2,This is a video i made of Black Hawk down duri...,50844,90.0,65.0,0


In [397]:
chart1 = (
    alt.Chart(SteamDB_merged_grouped)
    .mark_line(point=False)
    .encode(
         x=alt.X(
            "month:T",
            timeUnit="yearmonth",
            title="Month",
            axis=alt.Axis(format="%b %Y", labelAngle=-45, grid=False)
        ),
        y=alt.Y("peak:Q", title="Playerbase"),
        tooltip=["month:T", "peak:Q"]
    )
    .properties(
        title="Counter-Strike Playerbase Over Time (SteamDB)",
        width=800,
        height=400
    )
)
chart1.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting
